In [7]:
import pyspark
import pandas as pd
from pyspark.sql import functions as F

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext()
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load("train_data .csv")


In [2]:
df.show()

+-------+-------+----------+------------+--------+---------------+-----+---------+---------+--------------+------------------------------+---------+-----------+-----------------+-------------+--------+----------+----------+---+---------+------+---------------+--------------+-----------+------------+----------------+------------+-------------------+------------------------+----------+----------+-------------------+------------------+-------+-----------+----+----+----+----+----+----+----------+
|loan_id|user_id|total_loan|year_of_loan|interest|monthly_payment|class|sub_class|work_type| employer_type|                      industry|work_year|house_exist|house_loan_status|censor_status|marriage|offsprings|issue_date|use|post_code|region|debt_loan_ratio|del_in_18month|scoring_low|scoring_high|pub_dero_bankrup|early_return|early_return_amount|early_return_amount_3mon|recircle_b|recircle_u|initial_list_status|earlies_credit_mon|  title|policy_code|  f0|  f1|  f2|  f3|  f4|  f5|is_default|
+---

In [3]:
df.printSchema() 

root
 |-- loan_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- total_loan: double (nullable = true)
 |-- year_of_loan: integer (nullable = true)
 |-- interest: double (nullable = true)
 |-- monthly_payment: double (nullable = true)
 |-- class: string (nullable = true)
 |-- sub_class: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- employer_type: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- work_year: string (nullable = true)
 |-- house_exist: integer (nullable = true)
 |-- house_loan_status: integer (nullable = true)
 |-- censor_status: integer (nullable = true)
 |-- marriage: integer (nullable = true)
 |-- offsprings: integer (nullable = true)
 |-- issue_date: string (nullable = true)
 |-- use: integer (nullable = true)
 |-- post_code: double (nullable = true)
 |-- region: integer (nullable = true)
 |-- debt_loan_ratio: double (nullable = true)
 |-- del_in_18month: double (nullable = true)
 |-- scoring_low: dou

In [6]:
df["loan_id","total_loan"].describe().show()

+-------+------------------+-----------------+
|summary|           loan_id|       total_loan|
+-------+------------------+-----------------+
|  count|            300000|           300000|
|   mean|      399478.20423|      14399.20875|
| stddev|230869.13514231675|8709.179953286352|
|    min|                 0|            500.0|
|    max|            799999|          40000.0|
+-------+------------------+-----------------+



In [12]:
count_num = 0
for i in range(0,41000,1000):
    temp_df = df.select(df.total_loan.between(i,i+999).alias("total_loan"))
    temp_df = temp_df.where("total_loan=1")
    count_num = temp_df.count()
    print("between %d and %d  total loan: %d"%(i,i+1000,count_num))


between 0 and 1000  total loan: 2
between 1000 and 2000  total loan: 4043
between 2000 and 3000  total loan: 6341
between 3000 and 4000  total loan: 9317
between 4000 and 5000  total loan: 10071
between 5000 and 6000  total loan: 16514
between 6000 and 7000  total loan: 15961
between 7000 and 8000  total loan: 12789
between 8000 and 9000  total loan: 16384
between 9000 and 10000  total loan: 10458
between 10000 and 11000  total loan: 27170
between 11000 and 12000  total loan: 7472
between 12000 and 13000  total loan: 20513
between 13000 and 14000  total loan: 5928
between 14000 and 15000  total loan: 8888
between 15000 and 16000  total loan: 18612
between 16000 and 17000  total loan: 11277
between 17000 and 18000  total loan: 4388
between 18000 and 19000  total loan: 9342
between 19000 and 20000  total loan: 4077
between 20000 and 21000  total loan: 17612
between 21000 and 22000  total loan: 5507
between 22000 and 23000  total loan: 3544
between 23000 and 24000  total loan: 2308
betwee

In [65]:
all_type = df.select(df.employer_type).distinct().toPandas()
all_type = all_type['employer_type'].tolist()
print(all_type)
la = []
lb = []
for i in all_type:
    temp_df = df.select(F.when(df.employer_type==i,1).otherwise(0).alias("employer_type"))
    temp_df = temp_df.where("employer_type=1")
    count_num = temp_df.count()
    la.append(i)
    lb.append(count_num)
    print("the class is %s    the number is %f"%(i,count_num/300000))
dt = {'公司类型':la, '类型占比':lb}
dt = pd.DataFrame(dt)
dt.to_csv("公司类型.csv")

['幼教与中小学校', '上市企业', '政府机构', '世界五百强', '高等教育机构', '普通企业']
the class is 幼教与中小学校    the number is 0.099983
the class is 上市企业    the number is 0.100127
the class is 政府机构    the number is 0.258153
the class is 世界五百强    the number is 0.053707
the class is 高等教育机构    the number is 0.033687
the class is 普通企业    the number is 0.454343


In [66]:
df.withColumn("total_money",df.year_of_loan*df.monthly_payment*12-df.total_loan).select('user_id','total_money').show()
dt=df.withColumn("total_money",df.year_of_loan*df.monthly_payment*12-df.total_loan).select('user_id','total_money').toPandas()
dt.to_csv("total_moeny.csv")

+-------+------------------+
|user_id|       total_money|
+-------+------------------+
|      0|            3846.0|
|      1|1840.6000000000004|
|      2|10465.600000000002|
|      3|1758.5200000000004|
|      4| 1056.880000000001|
|      5| 7234.639999999999|
|      6| 757.9200000000001|
|      7| 4186.959999999999|
|      8|2030.7600000000002|
|      9|378.72000000000116|
|     10| 4066.760000000002|
|     11|1873.5599999999977|
|     12| 5692.279999999999|
|     13|1258.6800000000003|
|     14|6833.5999999999985|
|     15| 9248.200000000004|
|     16| 6197.119999999995|
|     17|1312.4400000000005|
|     18| 5125.200000000001|
|     19|1215.8400000000001|
+-------+------------------+
only showing top 20 rows



In [67]:
from pyspark.sql.functions import col
all_type = df.select(df.work_year).distinct().toPandas()['work_year'].tolist()
all_type.remove(None)
all_type.remove('5 years')
all_type.remove('1 year')
all_type.remove('2 years')
all_type.remove('4 years')
all_type.remove('3 years')
all_type.remove('< 1 year')
print(all_type)

temp_df = df.filter(col("work_year").isin(all_type))
temp_df.select('user_id','censor_status','work_year').show()
dt = temp_df.select('user_id','censor_status','work_year').toPandas()
dt.to_csv("censor.csv")
# all_type = df.select(df.censor_status).distinct().toPandas()['censor_status'].tolist()
# print(all_type)
# total_num = temp_df.count()
# s_temp_df = temp_df.where("employer_type=1")
# num1 = s_temp_df.count()/total_num

# temp_df = df.select(F.when(df.employer_type==1,1).when(df.employer_type==2,2).when(df.employer_type==3,3).alias("employer_type"))

['9 years', '7 years', '8 years', '6 years', '10+ years']
+-------+-------------+---------+
|user_id|censor_status|work_year|
+-------+-------------+---------+
|      1|            2|10+ years|
|      2|            1|10+ years|
|      5|            2|10+ years|
|      6|            0|  8 years|
|      7|            2|10+ years|
|      9|            0|10+ years|
|     10|            2|10+ years|
|     15|            1|  7 years|
|     16|            2|10+ years|
|     17|            0|10+ years|
|     18|            1|10+ years|
|     20|            1|  7 years|
|     21|            2|10+ years|
|     25|            2|10+ years|
|     26|            0|10+ years|
|     30|            0|10+ years|
|     31|            0|  6 years|
|     33|            1|10+ years|
|     38|            0|10+ years|
|     39|            1|10+ years|
+-------+-------------+---------+
only showing top 20 rows

